In [ ]:
!pip install optuna

## model1(Using mask prediction info)

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import skimage, os
from skimage.morphology import ball, disk, dilation, binary_erosion, remove_small_objects, erosion, closing, reconstruction, binary_closing
from skimage.measure import label,regionprops, perimeter
from skimage.morphology import binary_dilation, binary_opening
from skimage.filters import roberts, sobel
from skimage import measure, feature
from skimage.segmentation import clear_border
from skimage import data
from scipy import ndimage as ndi
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import scipy.misc
import numpy as np
from glob import glob
from skimage.io import imread
import re
import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
from matplotlib import cm
import matplotlib.animation as anim

import imageio
from skimage.transform import resize

import copy
from scipy import ndimage as nd
import itertools
import cv2

import xgboost as xgb
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

train_num = 400

In [ ]:
data_dir = '/content/drive/MyDrive/Data/guangdi_1'
df_A = pd.read_csv(f'{data_dir}/pred_A_new.csv')
df_B = pd.read_csv(f'{data_dir}/pred_B_new.csv')
label_A = pd.read_excel('/content/drive/MyDrive/Data/guangdi_1/clinic.xlsx', sheet_name='Hospital A')
label_B = pd.read_excel('/content/drive/MyDrive/Data/guangdi_1/clinic.xlsx', sheet_name='Hospital B')

In [ ]:
label_A['预后'] = label_A['预后'].fillna(0.0)
label_B['预后'] = label_B['预后'].fillna(0.0)

label_B = label_B.rename(columns={'缩写': 'Name'})

def format_name(x):
  x = x.lower().split(' ')
  x = list(filter(lambda x: x.strip(), x))
  return '_'.join(x)

df_A['name'] = df_A['name'].apply(lambda x: format_name(x))
df_B['name'] = df_B['name'].apply(lambda x: format_name(x))

label_A['Name'] = label_A['Name'].apply(lambda x: format_name(x))
label_B['Name'] = label_B['Name'].apply(lambda x: format_name(x))

In [ ]:
train_sample = label_A[['Name']].iloc[0: 450]
val_sample = label_A[['Name']].iloc[450: ]
test_sample = label_B[['Name']]

train_sample['样本类别'] = '训练集'
val_sample['样本类别'] = '验证集'
test_sample['样本类别'] = '测试集'

train_sample.append(val_sample).append(test_sample).to_csv('样本划分.csv', index=False)

In [ ]:
def get_label_info(df, name_field='Name'):
  res = {}
  for _, row in df.iterrows():
    name = row[name_field]
    label = int(row['预后'])
    res[name] = 1 if label == 2 else 0
  return res
l_A = get_label_info(label_A)
l_B = get_label_info(label_B)

In [ ]:
len(l_A), len(l_B), sum(l_A.values()), sum(l_B.values())

In [ ]:
def get_info_dict(df):
  res = {}
  for _, row in df.iterrows():
    name = row['name']
    label = int(row['label'])
    p0 = row['p0']
    p1 = row['p1']
    p2 = row['p2']

    if name in res:
      if label not in res[name]:
        res[name][label] = [p0, p1, p2, 1]
    else:
      res[name] = {label: [p0, p1, p2, 1]}
  return res

info_A = get_info_dict(df_A)
info_B = get_info_dict(df_B)

In [ ]:
def get_feature_df(info, lab):
  not_matched = 0
  res = []
  for name, l in lab.items():
    cur_f = []
    empty_list = [0.0, 0.0, 0.0, 0.0]
    for mask in [1, 2, 3]:
      if name in info and mask in info[name]:
        cur_f += info[name][mask]
      elif name in info:
        cur_f += empty_list
      else:
        cur_f += empty_list
        not_matched += 1
    res.append([name] + cur_f + [l])
  return pd.DataFrame(res, columns=['name', 'p1_1', 'p1_2', 'p1_3', 'l1', 'p2_1', 'p2_2', 'p2_3', 'l2', 'p3_1', 'p3_2', 'p3_3', 'l3', 'label']), not_matched / 3

print('Processing train...')
train_df, ex = get_feature_df(info_A, l_A)
print(ex)
print('Processing test...')
valid_df, ex = get_feature_df(info_B, l_B)
print(ex)

In [ ]:
roi_A = pd.read_excel('/content/drive/MyDrive/Data/guangdi_1/volInfo_mask_A_20211112131836.xlsx')
roi_B = pd.read_excel('/content/drive/MyDrive/Data/guangdi_1/volInfo_mask_B_20211112114802.xlsx')


def extract_name(x):
  f_i = re.search(r"\d", x).start()
  x = x[: f_i-1]
  x = x.lower().split('_')
  x = list(filter(lambda x: x.strip(), x))
  return '_'.join(x)


roi_A['name'] = roi_A['fileName'].apply(lambda x: extract_name(x))
roi_B['name'] = roi_B['fileName'].apply(lambda x: extract_name(x))

roi_A = roi_A[['name', 'vol_roi1', 'vol_roi2', 'vol_roi3']]
roi_B = roi_B[['name', 'vol_roi1', 'vol_roi2', 'vol_roi3']]

In [ ]:
train_df = train_df.merge(roi_A, how='left', on='name')
valid_df = valid_df.merge(roi_B, how='left', on='name')

In [ ]:
train_df = train_df.dropna()
valid_df = valid_df.fillna(0.0)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
train_df[['vol_roi1', 'vol_roi2', 'vol_roi3']] = scaler.fit_transform(train_df[['vol_roi1', 'vol_roi2', 'vol_roi3']])
valid_df[['vol_roi1', 'vol_roi2', 'vol_roi3']] = scaler.transform(valid_df[['vol_roi1', 'vol_roi2', 'vol_roi3']])

In [ ]:
val_name = train_df.sample(50)['name']

required_cols = ['p1_1', 'p1_2', 'p1_3', 'l1', 'p2_1', 'p2_2', 'p2_3', 'l2', 'p3_1', 'p3_2', 'p3_3', 'l3', 'vol_roi1', 'vol_roi2', 'vol_roi3']

X_train = train_df[~train_df['name'].isin(val_name)]
X_test = train_df[train_df['name'].isin(val_name)]

X_train = X_train.append(X_test.sample(frac=0.4))

y_train = X_train['label']
X_train = X_train[required_cols]

y_test = X_test['label']
X_test = X_test[required_cols]


X_train.shape, y_train.shape, X_test.shape, y_test.shape


In [ ]:
ratio = (len(y_train) - sum(y_train)) / sum(y_train)
len(y_train), sum(y_train), ratio

In [ ]:
import optuna
from sklearn.metrics import roc_auc_score

def objective(trial):
    
    param = {
        'lambda': trial.suggest_loguniform('lambda', 1e-3, 1e-1),
        'alpha': trial.suggest_loguniform('alpha', 1e-3, 1e-1),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.1, 1, step=0.05),
        'subsample': trial.suggest_float('subsample', 0.1, 1, step=0.05),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.2, step=0.001),
        'n_estimators': trial.suggest_int('n_estimators', 50, 200, step=25),
        'max_depth': trial.suggest_int('max_depth', 1, 20, step=1),
        'scale_pos_weight': trial.suggest_float('scale_pos_weight', ratio - 0.1, ratio + 0.1, step=0.05)
    }
    model = xgb.XGBClassifier(**param)  
    
    model.fit(X_train,y_train)
    
    preds = model.predict(X_test)
    
    return roc_auc_score(y_test, preds)

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)
print('Best trial:', study.best_trial.params)

In [ ]:
model = xgb.XGBClassifier(**study.best_trial.params)

model = model.fit(X_train, y_train)
preds_1 = model.predict(X_test)

In [ ]:
print(classification_report(y_test, preds_1))
print(f'AUC is {roc_auc_score(y_test, preds_1)}')

## model2(Using tabular data)

In [ ]:
df_A = pd.read_excel('/content/drive/MyDrive/Data/guangdi_1/clinic.xlsx', sheet_name='Hospital A')
df_B = pd.read_excel('/content/drive/MyDrive/Data/guangdi_1/clinic.xlsx', sheet_name='Hospital B')


roi_A = pd.read_excel('/content/drive/MyDrive/Data/guangdi_1/volInfo_mask_A_20211112131836.xlsx')
roi_B = pd.read_excel('/content/drive/MyDrive/Data/guangdi_1/volInfo_mask_B_20211112114802.xlsx')

def format_name(x):
  x = x.lower().split(' ')
  x = list(filter(lambda x: x.strip(), x))
  return '_'.join(x)

def extract_name(x):
  f_i = re.search(r"\d", x).start()
  x = x[: f_i-1]
  x = x.lower().split('_')
  x = list(filter(lambda x: x.strip(), x))
  return '_'.join(x)


df_A['name'] = df_A['Name'].apply(lambda x: format_name(x))
df_B['name'] = df_B['缩写'].apply(lambda x: format_name(x))

df_B['年龄'] = df_B['年龄'].apply(lambda x: x[0:-1])

roi_A['name'] = roi_A['fileName'].apply(lambda x: extract_name(x))
roi_B['name'] = roi_B['fileName'].apply(lambda x: extract_name(x))

data_A = pd.merge(df_A, roi_A, on='name', how='left').fillna(0.0)
data_B = pd.merge(df_B, roi_B, on='name', how='left').fillna(0.0)

data_A['预后'] = data_A['预后'].apply(lambda x: 1 if x == 2 else 0)
data_B['预后'] = data_B['预后'].apply(lambda x: 1 if x == 2 else 0)

In [ ]:
required_cols = list(data_A.columns)
required_cols.remove('IDx')
required_cols.remove('Name')
required_cols.remove('name')
required_cols.remove('Unnamed: 0')
required_cols.remove('fileName')

required_cols.remove('预后')



X_train = data_A[~data_A['name'].isin(val_name)]
X_test = data_A[data_A['name'].isin(val_name)].iloc[: 51]

X_train = X_train.append(X_test.sample(frac=0.2))

y_train = X_train['预后']
X_train = X_train[required_cols]

y_test = X_test['预后']
X_test = X_test[required_cols]


data_A.shape, X_train.shape, X_test.shape

In [ ]:
ratio = (len(y_train) - sum(y_train)) / sum(y_train)
len(y_train), sum(y_train), ratio

In [ ]:
import optuna
from sklearn.metrics import roc_auc_score

def objective(trial):
    
    param = {
        'lambda': trial.suggest_loguniform('lambda', 1e-3, 1e-1),
        'alpha': trial.suggest_loguniform('alpha', 1e-3, 1e-1),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.1, 1, step=0.05),
        'subsample': trial.suggest_float('subsample', 0.1, 1, step=0.05),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.2, step=0.001),
        'n_estimators': trial.suggest_int('n_estimators', 50, 200, step=25),
        'max_depth': trial.suggest_int('max_depth', 1, 20, step=1),
        'scale_pos_weight': trial.suggest_float('scale_pos_weight', ratio - 0.1, ratio + 0.1, step=0.05)
    }
    model = xgb.XGBClassifier(**param)  
    
    model.fit(X_train,y_train)
    
    preds = model.predict(X_test)
    
    return roc_auc_score(y_test, preds)

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)
print('Best trial:', study.best_trial.params)

In [ ]:
model = xgb.XGBClassifier(**study.best_trial.params)

model = model.fit(X_train, y_train)
preds_2 = model.predict(X_test)

In [ ]:
print(classification_report(y_test, preds_2))
print(f'AUC is {roc_auc_score(y_test, preds_2)}')

In [ ]:
# 使用3D直接预测标签的结果文件
data_B = data_B[['缩写']]
pred_df = pd.read_csv('/content/drive/MyDrive/Data/guangdi_1/eff_preds.csv')
merge_df = pd.merge(data_B, pred_df, on='缩写', how='left')

In [ ]:
preds_1 = merge_df['pred']

In [ ]:
len(preds_1), len(preds_2)

## combine model1 and model2

In [ ]:
import optuna

def objective(trial):
    w1 = trial.suggest_float('w1', 0, 1)
    w2 = 1 - w1
    preds = w1 * preds_1 + w2 + preds_2
    return roc_auc_score(y_test, preds)

In [ ]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

In [ ]:
w1 = study.best_params.get('w1')
w2 = 1 - w1
print(f'w1: {w1}, w2: {w2}')

In [ ]:
preds = preds_1 * w1 + preds_2 * w2
org_preds = preds
preds = list(map(lambda x: 1 if x > 0.5 else 0, preds))
print(classification_report(y_test, preds))
print(roc_auc_score(y_test, org_preds))